# Daily Results

## Initialisation

Basic approach to determine the project directory

In [1]:
import os
import sys
import glob
import datetime
import time

import csv
import json

In [2]:
projdir = os.path.realpath(os.path.join(sys.path[0], '..'))

configPath = os.path.join(projdir, 'config')

In [3]:
defaultNationality = 'England'

## Generic Class

Generic class to ensure that all custom classes are printable

In [4]:
class Printable:
    def __repr__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

## Entrant Class

Class to manage entrants

In [5]:
class Entrant(Printable):
    def __init__(self, entrantId, entrantDict, iswcRequired=False):
        '''Initialise entrant object'''

        # Entrant ID is generated by the calling routine
        self.entrantId = entrantId
        
        # Retain original dictionary for later use by reports and filters
        self.entrantDict = entrantDict
        
        # Quickly check if entrant details as defined in the config
        for key in config['Entrants']:
            if key != 'ISWC Member' or iswcRequired:
                # Check the attribute is present
                if key not in entrantDict:
                    raise ValueError('Missing "{}" for entrant #{}'.format(key, entrantId))
                # Check the attribute is valid
                if entrantDict[key] not in config['Entrants'][key]:
                    raise ValueError('Unexpected "{}" for entrant #{} - {}'.format(key, entrantId, entrantDict[key]))

        # Record personal details
        self.firstName = entrantDict['First Name']
        self.familyName = entrantDict['Family Name']
        self.gender = entrantDict['Gender']

        # Record nationality
        try:
            self.nationality = entrantDict['Nationality']
        except:
            self.nationality = defaultNationality
            
        # Record fleet details
        self.craftType = entrantDict['Craft Type']
        self.status = entrantDict['Status']
        self.entryType = entrantDict['EntryType']
        self.firstTimer = entrantDict['First Timer']

        # Record membership details
        self.ukwaMember = entrantDict['UKWA Member']
        if iswcRequired:
            self.iswcMember = entrantDict['ISWC Member']

## Event Class

Class to manage events

In [6]:
class Event():
    def __init__(self, path):
        self.path = path
        self.year = int(os.path.basename(path))
        self.entrants = []
        
        if self.year in config['Series']['ISWC']['Years']:
            self.iswcYear = True
        else:
            self.iswcYear = False
        
    def readEntrants(self):
        '''Read entrants from JSON'''

        filename = os.path.join(self.path, 'config', 'entrants.json')
        with open(filename, 'r', encoding='utf-8') as f:
            jsonTxt = f.read()
            entrantDicts = json.loads(jsonTxt)
            
            for entrantDict in entrantDicts:
                entrantId = len(self.entrants)
                entrant = Entrant(entrantId, entrantDict, iswcRequired=self.iswcYear)
                self.entrants.append(entrant)
                
    def summariseEntrants(self):
        '''Print summary of entrants'''
        
        print('{} entrants loaded'.format(len(self.entrants)))

## Process Years

Process all available years

In [7]:
pc1 = time.perf_counter()

In [8]:
# Read main config
filename = os.path.join(configPath, 'config.json')
with open(filename, 'r', encoding='utf-8') as f:
    jsonTxt = f.read()
    config = json.loads(jsonTxt)

In [9]:
# Only process the current year (for now)
eventPaths = sorted(glob.glob(os.path.join(projdir, '20[0-9][0-9]')))
year = datetime.datetime.now().year
for eventPath in eventPaths:
    if int(os.path.basename(eventPath)) == year:
        event = Event(eventPath)
        event.readEntrants()  
        event.summariseEntrants()

50 entrants loaded


In [10]:
pc2 = time.perf_counter()
print("Reports completed in %0.2f seconds" % (pc2 - pc1))

Reports completed in 0.17 seconds


## All Done!